# <b>Sliding Window Analysis and Feature Extraction</b>


In this version we will update a notebook by integrating the following comment:

➡️ In the context of activity recognition, **standardizing** the data is generally a good practice, especially if you are using machine learning algorithms that are sensitive to the scale of the input features. Standardization helps in ensuring that each feature contributes equally to the distance calculations and model training.

➡️ Use the other recorded data (**DMgravity** and acceleration raw data)

➡️ **Optimization** : noise filter and sampling window (2.56 s)

➡️ Keep the data related to the user `UAMSWH`

➡️ **Jerk signal**: the derivative of the acceleration signal. It quantifies how quickly the acceleration is changing over time.
We might apply a smoothing filter to the jerk signals to reduce noise (moving average/ Gaussian filters)


➡️ **Magnitude**: calculation of the Euclidean norm (or Euclidean length) of a vector. For a 3-dimensional signal, this means calculating the combined effect of the signal's components along the **X**, **Y**, and **Z** axes. This is particularly useful because it provides a single scalar value representing the overall magnitude of the vector at each time point, simplifying analysis and feature extraction

➡️ Run `tsne` without feature extraction

## **Steps:**

We will use data from only one user and one movement 


1. Determine the Window and Step Size:

    - Sampling rate: 20 Hz
    - Typical window size for capturing meaningful motion (e.g., 1 second): 20 samples
    - Step size with 50% overlap: 10 samples.
    
    <br>
    
2. Extract Time-Domain and Frequency-Domain Features:

    - Time-Domain: Mean, standard deviation, min, max, range
    - Frequency-Domain: Total power, dominant frequency, power in specific bands
    
    <br>

3. Visualization and Feature Extraction

## Import packages

In [4]:
import os
from time import time
from datetime import timedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

# Discrete Fourier transform of real or complex sequence.
from scipy.fftpack import fft

## Load data

In [6]:
# Storing the path of the extracted "train" folder 
Rehab_data_dir = "D:/Dropbox/Work/Learning/Python/Rehab data/"
HemiPhysioData_folder_name = "Sensor Data_ML scripts_Models/SensorData_HemiPhysioDataApp(Our App)/"

# Loading the train data
#data_train = pd.read_csv(Rehab_data_dir + HemiPhysioData_folder_name + "ml_data/train/u01/u01_movements_el-exfl_d_rt_20_20200520-2158.csv")
data_train = pd.read_csv(Rehab_data_dir + HemiPhysioData_folder_name + "Research 2/all_train.csv")

# Shape of train, test, and validation data
print('Shape of train data:', data_train.shape)

Shape of train data: (188717, 40)


In [7]:
# Columns to convert to float64
float_cols = [
    'AccelroX', 'AceelroY', 'AceelroZ', 'DMGrvX', 'DMGrvY', 'DMGrvZ',
    'DMPitch', 'DMQuatW', 'DMQuatX', 'DMQuatY', 'DMQuatZ',
    'DMRoll', 'DMRotX', 'DMRotY', 'DMRotZ',
    'DMUAccelX', 'DMUAccelY', 'DMUAccelZ',
    'DMYaw', 'TimeStamp'
]

# Columns to convert to int64
int_cols = ['Hertz', 'RecNo']

# Verify column names
print("Columns to convert to float:", float_cols)
print("Columns to convert to int:", int_cols)


Columns to convert to float: ['AccelroX', 'AceelroY', 'AceelroZ', 'DMGrvX', 'DMGrvY', 'DMGrvZ', 'DMPitch', 'DMQuatW', 'DMQuatX', 'DMQuatY', 'DMQuatZ', 'DMRoll', 'DMRotX', 'DMRotY', 'DMRotZ', 'DMUAccelX', 'DMUAccelY', 'DMUAccelZ', 'DMYaw', 'TimeStamp']
Columns to convert to int: ['Hertz', 'RecNo']


In [8]:
# Convert specified columns to float64
for col in float_cols:
    data_train[col] = pd.to_numeric(data_train[col], errors='coerce')

print("\nData Info After Converting to float:")
print(data_train.info())



Data Info After Converting to float:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188717 entries, 0 to 188716
Data columns (total 40 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   AM           188717 non-null  object 
 1   AMS          188717 non-null  object 
 2   AMSW         188717 non-null  object 
 3   AMW          188717 non-null  object 
 4   AccelroX     188717 non-null  float64
 5   AceelroY     188717 non-null  float64
 6   AceelroZ     188717 non-null  float64
 7   Activity     188717 non-null  object 
 8   DMGrvX       188717 non-null  float64
 9   DMGrvY       188717 non-null  float64
 10  DMGrvZ       188717 non-null  float64
 11  DMPitch      188717 non-null  float64
 12  DMQuatW      188717 non-null  float64
 13  DMQuatX      188717 non-null  float64
 14  DMQuatY      188717 non-null  float64
 15  DMQuatZ      188717 non-null  float64
 16  DMRoll       188717 non-null  float64
 17  DMRotX       188717 non-null 

In [9]:
# Convert specified columns to Int64 (nullable integer type)
for col in int_cols:
    data_train[col] = pd.to_numeric(data_train[col], errors='coerce').astype('Int64')

print("\nData Info After Converting to int:")
print(data_train.info())



Data Info After Converting to int:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188717 entries, 0 to 188716
Data columns (total 40 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   AM           188717 non-null  object 
 1   AMS          188717 non-null  object 
 2   AMSW         188717 non-null  object 
 3   AMW          188717 non-null  object 
 4   AccelroX     188717 non-null  float64
 5   AceelroY     188717 non-null  float64
 6   AceelroZ     188717 non-null  float64
 7   Activity     188717 non-null  object 
 8   DMGrvX       188717 non-null  float64
 9   DMGrvY       188717 non-null  float64
 10  DMGrvZ       188717 non-null  float64
 11  DMPitch      188717 non-null  float64
 12  DMQuatW      188717 non-null  float64
 13  DMQuatX      188717 non-null  float64
 14  DMQuatY      188717 non-null  float64
 15  DMQuatZ      188717 non-null  float64
 16  DMRoll       188717 non-null  float64
 17  DMRotX       188717 non-null  f

In [10]:
# Summary of missing values
missing_values = data_train.isnull().sum()
print("\nMissing Values per Column:")
print(missing_values[missing_values > 0])


Missing Values per Column:
GyroX    21454
GyroY    21454
GyroZ    21454
dtype: int64


In [11]:
data_train.dropna(inplace=True)

Window and Step Size:

- Sampling rate: 20 Hz
- Typical window size for capturing meaningful motion (e.g., 1 second): 20 samples
- Step size with 50% overlap: 10 samples.

Data to be Extracted:
- **DMRoll/Pitch/Yaw** - Device Motion processed values Roll, Pitch, and Yaw
- **DMRotX/Y/Z** - Device motion processed rotation x, y, and z 
- **DMUAccelX/Y/Z** - Device motion processed user acceleration x, y, and z. ***Excludes gravity, only includes motion.***
- **DMGrvX/Y/Z** - Device motion processed gravity x, y, and z


*More details on the notebook: `\Rehab data\Sensor Data_ML scripts_Models\SensorData_HemiPhysioDataApp(Our App)\ml_data`*

## Sliding Window Parameters

In [14]:
# Parameters 1
# sampling_rate = 20  # 20 Hz
# window_size = 20  # 1 second
# step_size = 10  # 50% overlap

# Parameters 2
sampling_rate = 20  # 20 Hz
window_size = 52  # 2.56 seconds (52 samples)
step_size = 26  # 50% overlap

## **Function to Calculate Jerk Signals and Magnitude**

We have to ensure that the jerk signals have the same length as the original signals by padding appropriately.

In [16]:
def moving_average(signal, window_size):
    return np.convolve(signal, np.ones(window_size)/window_size, mode='same')

def calculate_jerk_signals_and_magnitude(df, x_col, y_col, z_col):
    # Jerk signal (derivative of acceleration)
    jerk_x = np.diff(df[x_col].values, n=1, axis=0)
    jerk_y = np.diff(df[y_col].values, n=1, axis=0)
    jerk_z = np.diff(df[z_col].values, n=1, axis=0)

    # Add a padding to match the length of the original data
    jerk_x = np.pad(jerk_x, (1, 0), 'constant', constant_values=(0, 0))
    jerk_y = np.pad(jerk_y, (1, 0), 'constant', constant_values=(0, 0))
    jerk_z = np.pad(jerk_z, (1, 0), 'constant', constant_values=(0, 0))

    # Smoothing the jerk signals
    jerk_x = moving_average(jerk_x, window_size=5)  # Applying moving average filter
    jerk_y = moving_average(jerk_y, window_size=5)  # Applying moving average filter
    jerk_z = moving_average(jerk_z, window_size=5)  # Applying moving average filter

    # Magnitude calculation (Euclidean norm)
    magnitude = np.sqrt(df[x_col]**2 + df[y_col]**2 + df[z_col]**2)
    jerk_magnitude = np.sqrt(jerk_x**2 + jerk_y**2 + jerk_z**2)

    # Ensure the lengths match
    assert len(jerk_x) == len(df)
    assert len(jerk_y) == len(df)
    assert len(jerk_z) == len(df)
    assert len(magnitude) == len(df)
    assert len(jerk_magnitude) == len(df)

    # Add to the dataframe
    df[f'{x_col}_Jerk'] = jerk_x
    df[f'{y_col}_Jerk'] = jerk_y
    df[f'{z_col}_Jerk'] = jerk_z
    df[f'{x_col.split("_")[0]}_Mag'] = magnitude
    df[f'{x_col.split("_")[0]}_Jerk_Mag'] = jerk_magnitude

    # Rename columns to desired format based on signal names
    
    # DMUAccelX/Y/Z
    if (x_col, y_col, z_col) == ('DMUAccelX', 'DMUAccelY', 'DMUAccelZ'):
        df.rename(columns={f'{x_col.split("_")[0]}_Mag': 'DMUAccel_Mag',
                           f'{x_col.split("_")[0]}_Jerk_Mag': 'DMUAccel_Jerk_Mag'}, inplace=True)
    # 'DMRoll', 'DMPitch', 'DMYaw'
    elif (x_col, y_col, z_col) == ('DMRoll', 'DMPitch', 'DMYaw'):
        df.rename(columns={f'{x_col.split("_")[0]}_Mag': 'DMOrientation_Mag',
                           f'{x_col.split("_")[0]}_Jerk_Mag': 'DMOrientation_Jerk_Mag'}, inplace=True)
    # DMRotX/Y/Z
    elif (x_col, y_col, z_col) == ('DMRotX', 'DMRotY', 'DMRotZ'):
        df.rename(columns={f'{x_col.split("_")[0]}_Mag': 'DMRot_Mag',
                           f'{x_col.split("_")[0]}_Jerk_Mag': 'DMRot_Jerk_Mag'}, inplace=True)
    # DMGrvX/Y/Z
    elif (x_col, y_col, z_col) == ('DMGrvX', 'DMGrvY', 'DMGrvZ'):
        df.rename(columns={f'{x_col.split("_")[0]}_Mag': 'DMGrv_Mag',
                           f'{x_col.split("_")[0]}_Jerk_Mag': 'DMGrv_Jerk_Mag'}, inplace=True)

    # DMQuatX/YZ
    elif (x_col, y_col, z_col) == ('DMQuatX', 'DMQuatY', 'DMQuatZ'):
        df.rename(columns={f'{x_col.split("_")[0]}_Mag': 'DMQuat_Mag',
                           f'{x_col.split("_")[0]}_Jerk_Mag': 'DMQuat_Jerk_Mag'}, inplace=True)
        
    return df

## **Extract time-domain and frequency-domain features**
- Create Function to extract time-domain and frequency-domain features**

In [18]:
# Import specific packages
from scipy.stats import skew, kurtosis
from scipy.fftpack import fft
from scipy.signal import find_peaks
from statsmodels.tsa.ar_model import AutoReg

In [19]:
def extract_features(window, signal_name):
    # Time-domain features
    mean_val = np.mean(window)
    std_dev = np.std(window)
    mad_val = np.median(np.abs(window - np.median(window)))
    min_val = np.min(window)
    max_val = np.max(window)
    range_val = max_val - min_val
    sma_val = np.sum(np.abs(window))
    energy_val = np.sum(window**2) / len(window)
    iqr_val = np.percentile(window, 75) - np.percentile(window, 25)
    
    eps = np.finfo(float).eps  # Small constant to avoid log2(0)
    
    # Auto-regression coefficients (order 4)
    ar_model = AutoReg(window, lags=4).fit()
    ar_coeffs = ar_model.params
    
    # Frequency-domain features
    freqs = fft(window)
    freqs = np.abs(freqs[:len(freqs)//2])  # Take the positive frequencies
    power = freqs ** 2  # Power spectrum
    total_power = np.sum(power)
    max_ind = np.argmax(power)  # dominant_frequency
    mean_freq = np.sum(freqs * power) / total_power if total_power > 0 else 0
    skewness_val = skew(freqs)
    kurtosis_val = kurtosis(freqs)
    
    features = {
        f'{signal_name}_mean': mean_val,
        f'{signal_name}_std_dev': std_dev,
        f'{signal_name}_mad': mad_val,
        f'{signal_name}_min': min_val,
        f'{signal_name}_max': max_val,
        f'{signal_name}_range': range_val,
        f'{signal_name}_sma': sma_val,
        f'{signal_name}_energy': energy_val,
        f'{signal_name}_iqr': iqr_val,
        f'{signal_name}_arCoeff1': ar_coeffs[1] if len(ar_coeffs) > 1 else 0,
        f'{signal_name}_arCoeff2': ar_coeffs[2] if len(ar_coeffs) > 2 else 0,
        f'{signal_name}_arCoeff3': ar_coeffs[3] if len(ar_coeffs) > 3 else 0,
        f'{signal_name}_arCoeff4': ar_coeffs[4] if len(ar_coeffs) > 4 else 0,
        f'{signal_name}_power': total_power,
        f'{signal_name}_maxInd': max_ind,
        f'{signal_name}_meanFreq': mean_freq,
        f'{signal_name}_skewness': skewness_val,
        f'{signal_name}_kurtosis': kurtosis_val,
    }
    
    return features

## **Sliding Window Analysis Function**

* Integrate the user information and MoveType into the sliding window analysis 
* We modified the sliding_window_analysis function to handle grouping by `UAMSWH` and `SessionID`

Steps:

1. Group Data by User and Session: We use `groupby` to process each user's session separately.
2. Sliding Window Analysis: We apply the sliding window analysis within each group.
3. Keep User Information: Ensure user information and MoveType are retained in the features DataFrame.

In [21]:
def sliding_window_analysis_optimized(df, signals):
    all_features = []

    # Iterate over each group of user sessions
    for name, group in df.groupby(['UAMSWH', 'SessionID']):
        indices = group.index
        
        # Create sliding windows for each signal
        sliding_windows = {signal_name: np.lib.stride_tricks.sliding_window_view(signal[indices], window_size)[::step_size]
                           for signal_name, signal in signals.items()}
        
        # Ensure all signals have the same number of windows
        num_windows = len(next(iter(sliding_windows.values())))
        
        for window_idx in range(num_windows):
            combined_features = {}
            for signal_name, windows in sliding_windows.items():
                window = windows[window_idx]
                features = extract_features(window, signal_name)
                combined_features.update(features)
            
            # Include user information and MoveType
            combined_features['UAMSWH'] = group['UAMSWH'].iloc[0]
            combined_features['SessionID'] = group['SessionID'].iloc[0]
            combined_features['MoveType'] = group['MoveType'].iloc[0]
            
            all_features.append(combined_features)
    
    return pd.DataFrame(all_features)

## <b>Perform Analysis</b>

#### **Add Calculated Signals to DataFrame**

In [24]:
# Add jerk signals and magnitudes to the data_train DataFrame
data_train = calculate_jerk_signals_and_magnitude(data_train, 'DMUAccelX', 'DMUAccelY', 'DMUAccelZ')
data_train = calculate_jerk_signals_and_magnitude(data_train, 'DMRoll', 'DMPitch', 'DMYaw')
data_train = calculate_jerk_signals_and_magnitude(data_train, 'DMRotX', 'DMRotY', 'DMRotZ')
data_train = calculate_jerk_signals_and_magnitude(data_train, 'DMGrvX', 'DMGrvY', 'DMGrvZ')
data_train = calculate_jerk_signals_and_magnitude(data_train, 'DMQuatX', 'DMQuatY', 'DMQuatZ')

In [25]:
# Display the first few rows to verify
print(data_train.info())

<class 'pandas.core.frame.DataFrame'>
Index: 167263 entries, 1400 to 188716
Data columns (total 65 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   AM                      167263 non-null  object 
 1   AMS                     167263 non-null  object 
 2   AMSW                    167263 non-null  object 
 3   AMW                     167263 non-null  object 
 4   AccelroX                167263 non-null  float64
 5   AceelroY                167263 non-null  float64
 6   AceelroZ                167263 non-null  float64
 7   Activity                167263 non-null  object 
 8   DMGrvX                  167263 non-null  float64
 9   DMGrvY                  167263 non-null  float64
 10  DMGrvZ                  167263 non-null  float64
 11  DMPitch                 167263 non-null  float64
 12  DMQuatW                 167263 non-null  float64
 13  DMQuatX                 167263 non-null  float64
 14  DMQuatY               

#### **Perform sliding window analysis**

In [27]:
# Define the signals dictionary
signals = {
    'DMUAccel_X': data_train['DMUAccelX'].values,
    'DMUAccel_Y': data_train['DMUAccelY'].values,
    'DMUAccel_Z': data_train['DMUAccelZ'].values,
    'DMUAccel_X_Jerk': data_train['DMUAccelX_Jerk'].values,
    'DMUAccel_Y_Jerk': data_train['DMUAccelY_Jerk'].values,
    'DMUAccel_Z_Jerk': data_train['DMUAccelZ_Jerk'].values,
    'DMUAccel_Mag': data_train['DMUAccel_Mag'].values,
    'DMUAccel_Jerk_Mag': data_train['DMUAccel_Jerk_Mag'].values,
    ###########
    'DMRoll': data_train['DMRoll'].values,
    'DMPitch': data_train['DMPitch'].values,
    'DMYaw': data_train['DMYaw'].values,
    'DMRoll_Jerk': data_train['DMRoll_Jerk'].values,
    'DMPitch_Jerk': data_train['DMPitch_Jerk'].values,
    'DMYaw_Jerk': data_train['DMYaw_Jerk'].values,
    'DMOrientation_Mag': data_train['DMOrientation_Mag'].values,
    'DMOrientation_Jerk_Mag': data_train['DMOrientation_Jerk_Mag'].values,
    ###########
    'DMRotX': data_train['DMRotX'].values,
    'DMRotY': data_train['DMRotY'].values,
    'DMRotZ': data_train['DMRotZ'].values,
    'DMRotX_Jerk': data_train['DMRotX_Jerk'].values,
    'DMRotY_Jerk': data_train['DMRotY_Jerk'].values,
    'DMRotZ_Jerk': data_train['DMRotZ_Jerk'].values,
    'DMRot_Mag': data_train['DMRot_Mag'].values,
    'DMRot_Jerk_Mag': data_train['DMRot_Jerk_Mag'].values,
    ###########
    'DMGrvX': data_train['DMGrvX'].values,
    'DMGrvY': data_train['DMGrvY'].values,
    'DMGrvZ': data_train['DMGrvZ'].values,
    'DMGrvX_Jerk': data_train['DMGrvX_Jerk'].values,
    'DMGrvY_Jerk': data_train['DMGrvY_Jerk'].values,
    'DMGrvZ_Jerk': data_train['DMGrvZ_Jerk'].values,
    'DMGrv_Mag': data_train['DMGrv_Mag'].values,
    'DMGrv_Jerk_Mag': data_train['DMGrv_Jerk_Mag'].values,
    ###########
    'DMQuatX': data_train['DMQuatX'].values,
    'DMQuatY': data_train['DMQuatY'].values,
    'DMQuatZ': data_train['DMQuatZ'].values,
    'DMQuatX_Jerk': data_train['DMQuatX_Jerk'].values,
    'DMQuatY_Jerk': data_train['DMQuatY_Jerk'].values,
    'DMQuatZ_Jerk': data_train['DMQuatZ_Jerk'].values,
    'DMQuat_Mag': data_train['DMQuat_Mag'].values,
    'DMQuat_Jerk_Mag': data_train['DMQuat_Jerk_Mag'].values
}

#### **Perform sliding window analysis**

In [29]:
# Perform sliding window analysis
import time
from datetime import datetime

# Start the timer

start_time = time.time()
formatted_time = datetime.fromtimestamp(start_time).strftime('%Y-%m-%d %H:%M:%S')

print("===================================")
print("THE PROCESS START AT "+ formatted_time)
print("...-------------------...")

combined_features = sliding_window_analysis_optimized(data_train, signals)

# Calculate the total time taken
total_time = time.time() - start_time

print("===================================")
print('"THE PROCESS TOOK :"', f"{total_time:.{2}f}", 'seconds', '  |', f"{total_time/60:.{2}f}", 'minutes')

THE PROCESS START AT 2025-09-19 19:15:22
...-------------------...


IndexError: index 167785 is out of bounds for axis 0 with size 167263

In [ ]:
# Display the resulting combined features DataFrame
combined_features.info()

In [ ]:
combined_features.shape

### <b>Save the combined features</b>

- **`combined_features`**  All the combined features

In [ ]:
# Define the directory path
dir_path = 'combined_features'

# Check if the directory exists, if not, create it
if not os.path.exists(dir_path):
    os.makedirs(dir_path)

# Replace the colons in the timestamp with another character that is valid in filenames
formatted_time = formatted_time.replace(':', '-')

# Retrieve the number of features to save it with the file name
nb_features = combined_features.shape[1]

### Save the data
# Save all the combined_features All the combined features
combined_features.to_csv(dir_path+'/combined_features_{}_sRate_{}_wSize_{}-{}.csv'.format(nb_features, sampling_rate, window_size, formatted_time), index=False)